简答题：
1. 如果你的训练集具有数百万个特征，那么可以使用哪种线性回归训练算法？

2. 如果你的训练集里特征的数值大小迥异，那么哪些算法可能会受到影响？受影响程度如何？你应该怎么做？

3. 训练逻辑回归模型时，梯度下降可能会卡在局部最小值中吗？

4. 如果你让它们运行足够长的时间，是否所有的梯度下降算法都能得出相同的模型？

5. 假设你使用批量梯度下降，并在每个轮次绘制验证误差。如果你发现验证错误持续上升，那么可能是什么情况？你该如何解决？

6. 当验证误差上升时立即停止小批量梯度下降是个好主意吗？

7. 哪种梯度下降算法（在我们讨论过的算法中）将最快地到达最佳解附近？哪个实际上会收敛？如何使其他的也收敛

8. 假设你正在使用多项式回归。绘制学习曲线后，你会发现训练误差和验证误差之间存在很大的差距。发生了什么？解决此问题的三种方法是什么？

9. 假设你正在使用岭回归，并且你注意到训练误差和验证误差几乎相等且相当高。你是否会说模型存在高偏差或高方差？你应该增加正则化超参数α还是减小它呢？

10. 为什么要使用：a.岭回归而不是简单的线性回归（即没有任何正则化）？b.Lasso而不是岭回归？c.弹性网络而不是Lasso回归？

11. 假设你要将图片分类为室外/室内和白天/夜间。你应该实现两个逻辑回归分类器还是一个softmax回归分类器？

1. 随机梯度下降 小批量梯度下降
2. 梯度下降,SVM ; 导致损失函数扁平，梯度下降很慢，svm会被数值特征主导；对特征进行标准化，或者缩放
3. 不会
4. 不会
5. 可能是模型过拟合； 可以增加轮次，增加数据
6. 不是
7. 随机梯度下降 ； 批量梯度下降 ； 对其他的采用学习率衰减策略
8. 模型过拟合；  1 正则化  2 降低高项式 3 增加训练数据
9.  会 ； 减小它
10. a 岭回归加入L2正则化，可以缓解过拟合；  b  lasso采用L1正则化 会使部分特征系数收缩至0 ； c 弹性网络结合 L1 和 L2 正则化，既能进行特征选择，又能保留相关特征
11. 两个逻辑回归分类器。


编程题：

In [183]:
# todo 编程题: 在不使用sklearn的情况下，仅使用Numpy，为softmax回归实现带早停的批量梯度下降，将它用于分类任务，
#  例如鸢尾花数据集  load_iris, 只用两个特征就可以："petal width (cm)", "petal length (cm)"
#  强调：除了读数据，其他全用numpy （包括分离测试+验证），不用sklearn

#  注意：
#  1. 要实现l2正则化
#  2. 除了数据读取，其他仅使用numpy，包括训练集+验证集分离，以及softmax预测 和 损失计算

In [57]:
from sklearn.datasets import load_iris
import numpy as np
iris = load_iris(as_frame=True)
X_iris = iris.data[["petal length (cm)", "petal width (cm)"]].values
y_iris = iris.target

In [4]:
X_iris.shape,y_iris.shape

((150, 2), (150,))

In [69]:
X_mean = np.mean(X_iris, axis=0)
X_std = np.std(X_iris, axis=0)
X_iris = (X_iris - X_mean) / X_std

In [70]:
X_train, y_train = X_iris[:120],y_iris[:120]
X_iris_test,y_iris_test = X_iris[120:],y_iris[120:]

In [77]:
class SoftmaxRegressionWithL2:
    def __init__(self, learning_rate=0.01, lambda_reg=0.1, epochs=100000):
        self.learning_rate = learning_rate
        self.lambda_reg = lambda_reg
        self.epochs = epochs
        self.theta = None
        self.biases = None


    def softmax(self,x):
        exp_x = np.exp(x - np.max(x,axis=1,keepdims=True))
        return exp_x / np.sum(exp_x,axis=1,keepdims=True)

    def compute_loss(self,y_true, y_pred):
        m = len(y_true)
        cross_entropy = -np.log(y_pred[np.arange(m), np.argmax(y_true, 1)])
        l2_loss = (self.lambda_reg / (2 * m)) * np.sum(self.theta ** 2)
        return np.mean(cross_entropy) + l2_loss
    def to_onehot(self,y,num_classes):
        m = y.shape[0]
        onehot = np.zeros((m, num_classes))
        onehot[np.arange(m), y] = 1
        return onehot
    def fit(self,X, y):
        m, n = X.shape
        num_classes = len(np.unique(y))
        y_onehot = self.to_onehot(y, num_classes)
        K = num_classes
        self.theta = np.random.randn(n, K) * 0.01
        self.bias = np.zeros((1, K))

        for epoch in range(self.epochs):
            z = X.dot(self.theta) + self.bias
            y_pred = self.softmax(z)

            loss = self.compute_loss(y_onehot, y_pred)
            if epoch % 1000 == 0:
                print(f"epoch: {epoch}, loss: {loss:.4f}")
            error = y_pred - y_onehot
            theta_a = X.T.dot(error) /m + (self.lambda_reg / m) * self.theta
            bias_a = np.mean(error,axis=0,keepdims=True)

            self.theta -= self.learning_rate * theta_a
            self.bias -= self.learning_rate * bias_a
    def predict(self,X):
        z = X.dot(self.theta) + self.bias
        y_pred = self.softmax(z)
        return np.argmax(y_pred,axis=1)


In [78]:
model = SoftmaxRegressionWithL2(learning_rate=0.01, lambda_reg=0.01, epochs=10000)
model.fit(X_train, y_train)

y_pred = model.predict(X_iris_test)
y_true = y_iris_test
accuracy = np.mean(y_pred == y_true)
print(f"测试集准确率: {accuracy:.4f}")

epoch: 0, loss: 1.0904
epoch: 1000, loss: 0.3466
epoch: 2000, loss: 0.2523
epoch: 3000, loss: 0.2059
epoch: 4000, loss: 0.1780
epoch: 5000, loss: 0.1593
epoch: 6000, loss: 0.1457
epoch: 7000, loss: 0.1355
epoch: 8000, loss: 0.1275
epoch: 9000, loss: 0.1211
测试集准确率: 0.8667
